In [40]:
file1 = open("cited_spans_full1.txt","r+") 
  
text = file1.read()
text = text.split('\n')

paper_ids, cts = [], []
for i in text:
    if i != '':
        p = i.split('@@')
        paper_ids.append(p[0])
        cts.append(p[1])
p

['X96-1031',
 ' While the current MatchPlus learning law has proven to be effective in encoding relationships between words, it is computationally intensive and requires multiple passes through the training corpus. As a consequence, they share a common context vector. In practice, however, this summation is performed only over words that cooccur with the target word stem i.']

In [14]:
#!pip install pyenchant
import enchant
d = enchant.Dict("en_US")

True

In [63]:
import string


cts_pct_english_word = []
index = 0
for ct in cts:
    split_cts = ct.split(' ')
    count_english = 0
    space_check = 0
    for i in split_cts:
        j = i.translate(str.maketrans('', '', string.punctuation))
        if j != '':
            if d.check(j):
                count_english += 1
        else: 
            space_check -= 1
    cts_pct_english_word.append(count_english/(space_check+len(split_cts)))
    index+=1
    if index % 500 == 0:
        print(index)
cts_pct_english_word[0:6]

500
1000
1500
2000
2500
3000
3500
4000


[0.9142857142857143,
 0.9857142857142858,
 0.9803921568627451,
 0.9090909090909091,
 1.0,
 0.9523809523809523]

In [43]:
#88% of cited text span words are real english words

sum(cts_pct_english_word)/len(cts_pct_english_word)

0.8829155950600983

In [54]:
import pandas as pd
dfcts = pd.DataFrame({'paper_id':paper_ids,
             'cited text spans':cts,
             'cts pct english':cts_pct_english_word})
dfcts#.sort_values('cts pct english')

,paper_id,cited text spans,cts pct english
0,A00-1004,Parallel texts have been used in a number of...,0.914286
1,A00-1006,If we want to make a conversation proceed smoo...,0.985714
2,A00-1008,The purpose of the Atlas project is to enlarg...,0.980392
3,A00-1009,In this paper we present a linguistically mot...,0.909091
4,A00-1011,One major goal of information extraction (IE)...,1.000000
...,...,...,...
4203,W99-0904,Development of electronic morphological resou...,0.968750
4204,W99-0905,Choosing the correct translation of a content...,0.979167
4205,W99-0909,In this paper we discuss a potential solutio...,0.944444
4206,X93-1018,In evaluating the state of technology for ext...,0.941176


In [46]:
# Define data directory
import os
DATA_DIR = "./data/aan_data"

#load our 10k data into a dataframe
papers = []
filenames = [] #keep a reference for later transformations

abstracts2, bodys2, cits2, files2 = [],[],[],[]

for root, dirs, files in os.walk(DATA_DIR):
    for f in files:
        fn = root+"/"+f
        if "abstract" in fn:            
            in_file = open(fn, 'r')
            file = in_file.readlines()

            new_list = [''.join(file[i*4:(i+1)*4]) for i in range(int(len(file)/4))]
            list_no_n = [item.replace('\n','').replace('- ','').replace("\'", "") for item in new_list]

            string = ''
            for item in list_no_n:
                string = string + item            
            abstracts2.append(string)
            files2.append(f.split('_')[1].split('.')[0])
            
        elif "body" in fn:
            in_file = open(fn, 'r')
            file = in_file.readlines()

            new_list = [''.join(file[i*4:(i+1)*4]) for i in range(int(len(file)/4))]
            
            list_no_n = [item.replace('\n','').replace('- ','').replace("\'", "") for item in new_list]
            #list_no_n = [item for item in new_list]
            string = ''
            for item in list_no_n:
                string = string + item            
            bodys2.append(string)
        
        elif "CS" in fn:
            in_file = open(fn, 'r')
            file = in_file.readlines()
            string = [''.join(file) for i in file]
            if len(string) > 0: 
                string = [''.join(file) for i in file][0].replace('\n','').replace('- ','').replace("\'", "")
            cits2.append(string)
            
        else:
            pass
        
        
        
        
    #    with open(fn) as jsonfile:
    #        d = json.load(jsonfile)
    #    papers.append(d)
    #    filenames.append(f)

df2 = pd.DataFrame({'paper_id':files2,'abstract':abstracts2,'body':bodys2,'citations':cits2})
df2
        

,paper_id,abstract,body,citations
0,A00-1004,A major obstacle to the construction ofa pro...,Parallel texts have been used in a number of...,A compilation of parallel texts offered in a s...
1,A00-1006,This paper proposes a way to improve the tran...,"Recently, various dialogue translation syste...",We plan to improve the accuracy obtained so fa...
2,A00-1008,This paper describes an application of APE (t...,The purpose of the Atlas project is to enlarg...,Computer dialogue is now used at production st...
3,A00-1009,In this paper we describe an implemented fram...,In this paper we present a linguistically mot...,"From this viewpoint, research on paraphrasing ..."
4,A00-1011,"This paper reports on a large-scale, end-toen...",One major goal of information extraction (IE)...,"For example, Aone and Ramos-Santacruz (2000) p..."
...,...,...,...,...
4268,W99-0904,We present in this paper an unsupervised meth...,Development of electronic morphological resou...,Next along the spectrum of orthographic simila...
4269,W99-0905,This paper presents an unsupervised method fo...,Choosing the correct translation of a content...,"For comparable corpora, previous bilingual sen..."
4270,W99-0909,In this paper we report on an unsupervised a...,In this paper we discuss a potential solutio...,Watkinson and Manandhar (1999) present an unsu...
4271,X93-1018,This paper presents results from a study comp...,In evaluating the state of technology for ext...,"It may be noted that ""correctly"" is a problema..."


In [55]:
df = df2.merge(dfcts, on='paper_id')
df

,paper_id,abstract,body,citations,cited text spans,cts pct english
0,A00-1004,A major obstacle to the construction ofa pro...,Parallel texts have been used in a number of...,A compilation of parallel texts offered in a s...,Parallel texts have been used in a number of...,0.914286
1,A00-1006,This paper proposes a way to improve the tran...,"Recently, various dialogue translation syste...",We plan to improve the accuracy obtained so fa...,If we want to make a conversation proceed smoo...,0.985714
2,A00-1008,This paper describes an application of APE (t...,The purpose of the Atlas project is to enlarg...,Computer dialogue is now used at production st...,The purpose of the Atlas project is to enlarg...,0.980392
3,A00-1009,In this paper we describe an implemented fram...,In this paper we present a linguistically mot...,"From this viewpoint, research on paraphrasing ...",In this paper we present a linguistically mot...,0.909091
4,A00-1011,"This paper reports on a large-scale, end-toen...",One major goal of information extraction (IE)...,"For example, Aone and Ramos-Santacruz (2000) p...",One major goal of information extraction (IE)...,1.000000
...,...,...,...,...,...,...
4203,W99-0904,We present in this paper an unsupervised meth...,Development of electronic morphological resou...,Next along the spectrum of orthographic simila...,Development of electronic morphological resou...,0.968750
4204,W99-0905,This paper presents an unsupervised method fo...,Choosing the correct translation of a content...,"For comparable corpora, previous bilingual sen...",Choosing the correct translation of a content...,0.979167
4205,W99-0909,In this paper we report on an unsupervised a...,In this paper we discuss a potential solutio...,Watkinson and Manandhar (1999) present an unsu...,In this paper we discuss a potential solutio...,0.944444
4206,X93-1018,This paper presents results from a study comp...,In evaluating the state of technology for ext...,"It may be noted that ""correctly"" is a problema...",In evaluating the state of technology for ext...,0.941176


In [64]:
ei = df[df['cts pct english'] < .8]
ei['cited text spans'][36]

'  The task of predicting the most likely word based on properties of its surrounding context is the archetypical prediction problem in natural language processing (NLP). will J  b~rd corped~%~ det pcomp det.. a-.. Vir~.n attr "moc l  tl~e director, Pierre years attr * non-eXecutive qnt 6t Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29. Def in i t ion 4 (Compos ing  features)  Let f l ,  f2 be feature definitions.'

In [57]:
#eliminating every paper with lower than 80% cts english words raises it to 90%
ei = df[df['cts pct english'] > .8]
sum(ei['cts pct english'])/len(ei['cts pct english'])

0.8964151754149946

In [62]:
#eliminating every paper with lower than 80% cts english words raises it to 90%
ei = df[df['cts pct english'] > .95]
#sum(ei['cts pct english'])/
len(ei['cts pct english'])

676